First, we use SQLAlchemy to setup a simple sqlite db:


In [36]:
from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer, select, column

engine = create_engine("sqlite:///:memory:",future=True)
metadata_obj = MetaData(bind=engine)


In [37]:
# create city SQL table
table_name = "city_stats"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
)
metadata_obj.create_all()

Now it’s time to insert some datapoints!

In [38]:
from sqlalchemy import insert
rows = [
    {"city_name": "Toronto", "population": 2731571, "country": "Canada"},
    {"city_name": "Tokyo", "population": 13929286, "country": "Japan"},
    {"city_name": "Berlin", "population": 600000, "country": "Germany"},
]
for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.connect() as connection:
        cursor = connection.execute(stmt)

Finally, we can wrap the SQLAlchemy engine with our SQLDatabase wrapper; this allows the db to be used within LlamaIndex:

In [39]:
from llama_index import SQLDatabase

sql_database = SQLDatabase(engine, include_tables=["city_stats"])

 In this section, we show how we can populate the city_stats table by ingesting Wikipedia articles about each city.

First, we use the Wikipedia reader from LlamaHub to load some pages regarding the relevant data.

In [41]:
from llama_index import download_loader

WikipediaReader = download_loader("WikipediaReader")
wiki_docs = WikipediaReader().load_data(pages=['Moscow','Toronto','Tokyo','Berlin'])

When we build the SQL index, we can specify these docs as the first input; these documents will be converted to structured datapoints and inserted into the db:

In [42]:
import os
os.environ["OPENAI_API_KEY"] = 'Enter-API-KEY'

from llama_index import GPTSQLStructStoreIndex, SQLDatabase

sql_database = SQLDatabase(engine, include_tables=["city_stats"])
# NOTE: the table_name specified here is the table that you
# want to extract into from unstructured documents.
index = GPTSQLStructStoreIndex.from_documents(
    wiki_docs, 
    sql_database=sql_database, 
    table_name="city_stats",
)

INFO:llama_index.indices.common.struct_store.base:> Adding chunk 0: Moscow ( MOS-koh, US chiefly  MOS-kow; Russian:...
INFO:llama_index.indices.common.struct_store.base:> Adding chunk 1: of the origin of the name of the river have bee...
INFO:llama_index.indices.common.struct_store.base:> Adding chunk 2: meaning both "forty, a great many" and "a distr...
INFO:llama_index.indices.common.struct_store.base:> Adding chunk 3: for many years and attracted a large number of ...
INFO:llama_index.indices.common.struct_store.base:> Adding chunk 4: width of 36 metres (118 feet), and a depth of 9...
INFO:llama_index.indices.common.struct_store.base:> Adding chunk 5: all of them being driven from their homeland to...
INFO:llama_index.indices.common.struct_store.base:> Adding chunk 6: on the platform of the Lobnoye mesto.
The road ...
INFO:llama_index.indices.common.struct_store.base:> Adding chunk 7: the political centre of the new state.
With the...
INFO:llama_index.indices.common.struct_store.bas

INFO:llama_index.indices.common.struct_store.base:> Adding chunk 11: transportation within Tokyo is dominated by an ...
INFO:llama_index.indices.common.struct_store.base:> Adding chunk 12: local wards or municipal offices. Most public s...
INFO:llama_index.indices.common.struct_store.base:> Adding chunk 13: (Fuchū) and Toshiba Brave Lupus Tokyo (Fuchū).
...
INFO:llama_index.indices.common.struct_store.base:> Adding chunk 14: several of Tokyo's medical and scientific facil...
INFO:llama_index.indices.common.struct_store.base:> Adding chunk 0: Berlin ( bur-LIN, German: [bɛʁˈliːn] (listen)) ...
INFO:llama_index.indices.common.struct_store.base:> Adding chunk 1: the most visited zoo in Europe and one of the m...
INFO:llama_index.indices.common.struct_store.base:> Adding chunk 2: of Moabit bears a French-derived name, and Fran...
INFO:llama_index.indices.common.struct_store.base:> Adding chunk 3: initiated a policy of promoting immigration and...
INFO:llama_index.indices.common.struct_store

you can take a look at the current table to verify that the datapoints have been inserted!

In [43]:
# view current table
stmt = select(
    [column("city_name"), column("population"), column("country")]
).select_from(city_stats_table)

with engine.connect() as connection:
    results = connection.execute(stmt).fetchall()
    print(results)

[('Moscow', 25000, 'Russia'), ('Toronto', 1000000, 'Canada'), ('Tokyo', 6100, 'Japan'), ('Berlin', 600000, 'Germany')]


In [46]:
# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine()
response = query_engine.query("Which city has the Miminum population?")
print(response)


INFO:llama_index.indices.struct_store.sql_query:> Table desc str: Schema of table city_stats:
Table 'city_stats' has columns: city_name (VARCHAR(16)), population (INTEGER), country (VARCHAR(16)) and foreign keys: .

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 252 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 0 tokens


[('Tokyo', 6100)]


In [47]:
response.extra_info['sql_query']

'SELECT city_name, MIN(population) \nFROM city_stats;'